# Работа с базой данных на основе Startup Investments

Нужно проанализировать данные о фондах и инвестициях и написать запросы к базе.

## Схема базы данных

![jupyter](./13_border2880_1628164784.png)

### 1. Посчитайте, сколько компаний закрылось.

SELECT COUNT(status)

FROM company

WHERE status = 'closed';

![jupyter](./1.png)

### 2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .


SELECT funding_total

FROM company

WHERE country_code = 'USA'

AND category_code = 'news'

ORDER BY funding_total DESC;

![jupyter](./2.png)

### 3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.


SELECT SUM(price_amount)

FROM acquisition

WHERE EXTRACT('year' FROM acquired_at::date) BETWEEN '2011' AND '2013'

AND term_code = 'cash';

![jupyter](./3.png)

### 4. Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.


SELECT first_name,
       last_name,
       twitter_username
       
FROM people

WHERE twitter_username LIKE 'Silver%';

![jupyter](./4.png)

### 5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

SELECT *

FROM people

WHERE twitter_username LIKE '%money%'

AND last_name LIKE 'K%';

![jupyter](./5.png)

### 6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.


SELECT SUM(funding_total),

        country_code
        
FROM company

GROUP BY country_code

ORDER BY SUM(funding_total) DESC;


![jupyter](./6.png)

### 7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату. Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

SELECT funded_at,

        MIN(raised_amount),
        
        MAX(raised_amount)
        
FROM funding_round

GROUP BY funded_at

HAVING MIN(raised_amount) <> 0

AND MIN(raised_amount) <> MAX(raised_amount);

![jupyter](./7.png)

### 8. Создайте поле с категориями: 
 * **Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.**
 * **Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.**
 * **Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.**
 
**Отобразите все поля таблицы fund и новое поле с категориями.**

SELECT *,

      CASE 
      
          WHEN invested_companies >= 100 THEN 'high_activity'
          
          WHEN invested_companies >= 20 THEN 'middle_activity'
          
          ELSE 'low_activity'
          
      END
      
FROM fund;


![jupyter](./8.png)

### 9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

SELECT ROUND(AVG(investment_rounds)),

       CASE
       
           WHEN invested_companies>=100 THEN 'high_activity'
           
           WHEN invested_companies>=20 THEN 'middle_activity'
           
           ELSE 'low_activity'
           
       END AS activity
       
FROM fund

GROUP BY activity

ORDER BY ROUND(AVG(investment_rounds));

![jupyter](./9.png)

### 10. Выгрузите таблицу с десятью самыми активными инвестирующими странами. Активность страны определите по среднему количеству компаний, в которые инвестируют фонды этой страны. Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды, основанные с 2010 по 2012 год включительно.

SELECT MIN(invested_companies),

       MAX(invested_companies),
       
       AVG(invested_companies),
       
       country_code
       
FROM fund

WHERE founded_at::DATE BETWEEN '2010-01-01' AND '2012-12-31'

GROUP BY country_code

HAVING MIN(invested_companies) <> 0

ORDER BY AVG(invested_companies) DESC, country_code

LIMIT 10;

![jupyter](./10.png)

### 11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

SELECT p.first_name,

       p.last_name,
       
       e.instituition
       
FROM people AS p

LEFT OUTER JOIN education AS e ON e.person_id = p.id;

![jupyter](./11.png)

### 12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.


SELECT DISTINCT c.name,

       COUNT(DISTINCT e.instituition)
       
FROM company AS c

JOIN people AS p ON c.id = p.company_id

LEFT OUTER JOIN education AS e ON e.person_id = p.id

GROUP BY c.name

ORDER BY COUNT(DISTINCT e.instituition) DESC

LIMIT 5;

![jupyter](./12.png)

### 13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

WITH

c AS (SELECT *
      
    FROM company
      
    WHERE status = 'closed'),

fr AS (SELECT *
       
     FROM funding_round
       
     WHERE is_first_round = 1 AND is_last_round = 1)

SELECT

   DISTINCT c.name
    
FROM c AS c

JOIN fr AS fr ON c.id = fr.company_id;

![jupyter](./13.png)

### 14.Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

WITH

c AS (SELECT *

    FROM company
    
    WHERE status = 'closed'),
    
fr AS (SELECT *,

     FROM funding_round
     
     WHERE is_first_round = 1 AND is_last_round = 1)
     
SELECT

   DISTINCT p.id
   
FROM c AS c

JOIN fr AS fr ON c.id = fr.company_id

JOIN people AS p ON c.id = p.company_id;

![jupyter](./14.png)

### 15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

WITH

c AS (SELECT *

    FROM company
    
    WHERE status = 'closed'),
    
fr AS (SELECT *

     FROM funding_round
     
     WHERE is_first_round = 1 AND is_last_round = 1)
     
SELECT

   DISTINCT p.id,
   
    ed.instituition
    
FROM c AS c

JOIN fr AS fr ON c.id = fr.company_id

JOIN people AS p ON c.id = p.company_id

JOIN education AS ed ON ed.person_id = p.id;

![jupyter](./15.png)

### 16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

SELECT

    DISTINCT person_id AS person_id,
    
    COUNT(instituition)
    
FROM education

 WHERE person_id IN(
 
SELECT

   DISTINCT id
   
FROM people

WHERE company_id IN (SELECT id
                    
                    FROM company
                    
                    WHERE id IN (SELECT DISTINCT company_id
                    
            FROM funding_round
            
     WHERE is_first_round = 1 AND is_last_round = 1
     
                                )
                                
                    AND status = 'closed'))
                    
GROUP BY person_id;

![jupyter](./16.png)

### 17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

SELECT AVG(inst)

FROM(

SELECT

    DISTINCT person_id AS person_id,
    
    COUNT(instituition) AS inst
    
FROM education

 WHERE person_id IN(
 
SELECT

   DISTINCT id
   
FROM people

WHERE company_id IN (SELECT

                    id
                    
                    FROM company
                    
                    WHERE id IN (SELECT DISTINCT company_id
                    
            FROM funding_round
            
     WHERE is_first_round = 1 AND is_last_round = 1
     
                                )
                                
                    AND status = 'closed'))
                    
GROUP BY person_id) AS i;

![jupyter](./17.png)

### 18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

SELECT AVG(count)

FROM

    (SELECT person_id,
    
           COUNT(instituition) AS count
           
    FROM company AS c
    
    JOIN people AS p ON c.id=p.company_id
    
    JOIN education AS e ON p.id=e.person_id
    
    WHERE c.name = 'Facebook'
    
    GROUP BY person_id) AS a;

![jupyter](./18.png)

### 19.Составьте таблицу из полей:
 * **name_of_fund — название фонда;**
 * **name_of_company — название компании;**
 * **amount — сумма инвестиций, которую привлекла компания в раунде.**
 
**В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.**

SELECT DISTINCT f.name AS name_of_fund,

                c.name AS name_of_company,
                
                fr.raised_amount AS amount
                
FROM investment i

JOIN company c ON i.company_id = c.id

JOIN fund f ON i.fund_id = f.id

JOIN funding_round fr ON i.funding_round_id = fr.id

WHERE c.milestones > 6

  AND extract(YEAR
  
              FROM fr.funded_at::date) BETWEEN 2012 AND 2013;

![jupyter](./19.png)

### 20. Выгрузите таблицу, в которой будут такие поля:
 * **название компании-покупателя;**
 * **сумма сделки;**
 * **название компании, которую купили;**
 * **сумма инвестиций, вложенных в купленную компанию;**
 * **доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.**
 
**Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.**

**Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.**

SELECT c.name AS acquiring_name,

       a.price_amount,
       
       c1.name AS acquired_name,
       
       c1.funding_total,
       
       ROUND(a.price_amount/c1.funding_total)
       
FROM acquisition AS a

LEFT JOIN company AS c ON a.acquiring_company_id = c.id

LEFT JOIN company AS c1 ON a.acquired_company_id = c1.id

WHERE a.price_amount > 0 AND c1.funding_total > 0

ORDER BY a.price_amount DESC, c1.name

LIMIT 10;

![jupyter](./20.png)

### 21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

SELECT name,

       EXTRACT(MONTH FROM fr.funded_at)
    
FROM company AS c

LEFT OUTER JOIN funding_round AS fr ON fr.company_id = c.id

WHERE fr.raised_amount <> 0

AND EXTRACT(YEAR FROM fr.funded_at::date) BETWEEN 2010 AND 2013

AND c.category_code = 'social';

![jupyter](./21.png)

### 22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
 * **номер месяца, в котором проходили раунды;**
 * **количество уникальных названий фондов из США, которые инвестировали в этом месяце;**
 * **количество компаний, купленных за этот месяц;**
 * **общая сумма сделок по покупкам в этом месяце.**

WITH ac AS (SELECT COUNT(acquired_company_id) AS count_acquired,

                      SUM(price_amount) AS summa,
                      
                      EXTRACT(MONTH FROM acquired_at) AS month
                      
              FROM acquisition
              
              WHERE DATE_TRUNC('year', acquired_at) BETWEEN '2010.01.01' AND '2013.12.31'
              
              GROUP BY EXTRACT(MONTH FROM acquired_at)),
              
     fur AS (SELECT EXTRACT(MONTH FROM funded_at) AS month,
     
                    COUNT(DISTINCT i.fund_id) AS count_fund
                    
             FROM funding_round AS fr
             
             FULL JOIN investment AS i ON i.funding_round_id= fr.id
             
             FULL JOIN fund AS f ON f.id=i.fund_id
             
             WHERE DATE_TRUNC('year', funded_at) BETWEEN '2010.01.01' AND '2013.12.31'
             
                   AND f.country_code='USA'
                   
             GROUP BY EXTRACT(MONTH FROM funded_at))
             
SELECT ac.month AS month,

       fur.count_fund AS funds,
       
       count_acquired AS companies,
       
       summa AS summa
       
FROM ac

FULL JOIN fur ON ac.month=fur.month

ORDER BY ac.month;

![jupyter](./22.png)

### 23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

WITH

y11 AS

   (SELECT country_code,
   
           AVG(funding_total) AS avg_2011
           
    FROM company
    
    WHERE EXTRACT(YEAR FROM founded_at)=2011
    
    GROUP BY country_code),
    
y12 AS

   (SELECT country_code,
   
           AVG(funding_total) AS avg_2012
           
    FROM company
    
    WHERE EXTRACT(YEAR FROM founded_at)=2012
    
    GROUP BY country_code),
    
y13 AS

   (SELECT country_code,
   
           AVG(funding_total) AS avg_2013
           
    FROM company
    
    WHERE EXTRACT(YEAR FROM founded_at)=2013
    
    GROUP BY country_code)
    
SELECT y11.country_code,

       y11.avg_2011,
       
       y12.avg_2012,
       
       y13.avg_2013
       
FROM y11

INNER JOIN y12 ON y11.country_code=y12.country_code

INNER JOIN y13 ON y11.country_code=y13.country_code

ORDER BY y11.avg_2011 DESC;

![jupyter](./23.png)